In [2]:
import os
import json
import pandas as pd
import traceback

# https://python.langchain.com/docs/integrations/chat/openai
from langchain.chat_models import ChatOpenAI

In [3]:
from dotenv import load_dotenv

load_dotenv()

Python-dotenv could not parse statement starting at line 1


True

In [4]:
APIKEY_OPENAI = os.getenv("APIKEY_OPENAI")

In [5]:
# Initialize ChatopenAI
llm = ChatOpenAI(openai_api_key=APIKEY_OPENAI, model_name="gpt-3.5-turbo",temperature=0.5)

/home/pjlinux/projects/mcqgen/env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [6]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import  SequentialChain
from langchain.callbacks import get_openai_callback

In [7]:
# JSON response for the chatbot
RESPONSE_JSON = {
    "1": {
        "mcq":"multiple choice question",
        "options":{"a":"choice here",
                   "b":"choice here",
                   "c":"choice here",
                   "d":"choice here"
                   },
        "correct":"correct_answer",
    },
    "2": {
        "mcq":"multiple choice question",
        "options":{"a":"choice here",
                   "b":"choice here",
                   "c":"choice here",
                   "d":"choice here"
                   },
        "correct":"correct_answer",
    },
    "3": {
        "mcq":"multiple choice question",
        "options":{"a":"choice here",
                   "b":"choice here",
                   "c":"choice here",
                   "d":"choice here"
                   },
        "correct":"correct_answer",
    },
    "4": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct_answer"
    },
    "5": {
        "mcq":"multiple choice question",
        "options":{"a":"choice here",
                   "b":"choice here",
                   "c":"choice here",
                   "d":"choice here"
                   },
        "correct":"correct_answer",
    }
}

In [8]:
# Template 1 for the prompt
TEMPLATE ="""
text:{text}
You are an expert MCQ maker. Give the above text, it is your job to\
create a quize of {number} multiple choice questions for {subject} in a {tone} tone\
Make sure questions are not repeated and check all question to be confirming to the text as well. \
Make sure to format your resepomse like RESPONSE_JSON below and use it as a guide.  \
Ensure to make {number} MCQs
###RESPONSE_JSON
{response_json}
"""


In [9]:
# Quiz generation prompt
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [10]:
# Quiz generation chain
quize_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [11]:
# Template 2 for the prompt
TEMPLATE2="""
Your are an expert English grammerian. Give a multiple choice quiz for {subject} students. \
    you need to evaluate the complexity of the question and give a complete analysis of the quiz. 
    Only use at max 50 words for the complexity analysis. \
    If the questions are not in par with the student's cognitive and analytical abilities of the student, \
    Update the questions which needs to be changed and change the tone such that it perfectly fits the student's cognitive and analytical abilities. \
    Quiz_MCQs: 
    {quiz}

    check from an expert English writer of the above quiz:
    """

In [12]:
# Question evaluation prompt
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE2
)

In [13]:
# Quiz evaluation chain
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [14]:
# Sequential chain for quiz generation and evaluation
generate_evaluate_chain = SequentialChain(  
    chains=[quize_chain, review_chain],
    input_variables=["text", "number", "subject", "tone", "response_json"],
    output_variables=["quiz", "review"],
    verbose=True
)

In [15]:
# File path for the Text data
file_path = r"/home/pjlinux/projects/mcqgen/data.txt"

In [16]:
# Read the TEXT data
with open(file_path, "r") as file:
    TEXT = file.read()

In [17]:
# Serialize python dictionary to JSON
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct_answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct_answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct_answer"}, "4": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct_answer"}, "5": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct_answer"}}'

In [18]:
NUMBER = 5
SUBJECT = "Production Possibility Frontier"
TONE = "difficult"

In [19]:
# https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

# How to use the token usage tracking
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
    {
        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)
    }
    )

/home/pjlinux/projects/mcqgen/env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

text:Demystifying Choices: Understanding the Production Possibility Curve
Imagine a world where you can have everything you desire. Unfortunately, reality presents limitations. In economics, the Production Possibility Curve (PPC) depicts this very concept – the trade-offs between producing two goods or services within a given timeframe and resource constraints.

Visualizing Scarcity: The Curve Explained

The PPC is a graph with two axes representing the production quantities of two goods (Good A and Good B). Each point on the curve signifies a feasible production combination achievable with the available resources (labor, capital, technology). Points inside the curve imply underutilization of resources, while points outside represent an unrealistic scenario exceeding resource capacity.

The PPC typically slopes downward from left to right, signifying the trade-off. To produce more of 

In [20]:
# Print token usage
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost (USD): ${cb.total_cost}")

Total Tokens: 1713
Prompt Tokens: 1338
Completion Tokens: 375
Total Cost (USD): $0.0027570000000000003


In [21]:
response

{'text': 'Demystifying Choices: Understanding the Production Possibility Curve\nImagine a world where you can have everything you desire. Unfortunately, reality presents limitations. In economics, the Production Possibility Curve (PPC) depicts this very concept – the trade-offs between producing two goods or services within a given timeframe and resource constraints.\n\nVisualizing Scarcity: The Curve Explained\n\nThe PPC is a graph with two axes representing the production quantities of two goods (Good A and Good B). Each point on the curve signifies a feasible production combination achievable with the available resources (labor, capital, technology). Points inside the curve imply underutilization of resources, while points outside represent an unrealistic scenario exceeding resource capacity.\n\nThe PPC typically slopes downward from left to right, signifying the trade-off. To produce more of Good A, you must sacrifice some production of Good B, and vice versa. This reflects the sca

In [22]:
quiz = response.get("quiz")

In [23]:
quiz = json.loads(quiz)

In [24]:
review = response.get("review")

In [25]:
print(review)

The quiz questions are well-structured and cover key concepts of the Production Possibility Frontier. The complexity level is appropriate for students studying this topic. The language used is clear and concise, making it easy for students to understand and answer the questions effectively.


In [26]:
# Create a table for the quiz data
quiz_table_data = []
for key, value in quiz.items():
    mcq = value.get("mcq")
    options = " | ".join(
        [
            f"{option} : {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "choices": options, "correct": correct})


In [27]:
quiz_table_data

[{'MCQ': 'What does a point inside the Production Possibility Curve signify?',
  'choices': 'a : Optimal resource utilization | b : Underutilization of resources | c : Unrealistic scenario | d : Efficiency frontiers',
  'correct': 'b'},
 {'MCQ': 'What does a curved Production Possibility Curve indicate?',
  'choices': 'a : Constant trade-off ratio | b : Diminishing marginal rate of transformation | c : Optimal resource allocation | d : Technological advancements',
  'correct': 'b'},
 {'MCQ': 'What can shift the Production Possibility Curve outwards?',
  'choices': 'a : Decreased efficiency | b : Resource scarcity | c : Technological advancements | d : Unrealistic scenarios',
  'correct': 'c'},
 {'MCQ': 'What does the Production Possibility Curve help visualize?',
  'choices': 'a : Resource abundance | b : Opportunity cost of production choices | c : Complete resource utilization | d : Static technology',
  'correct': 'b'},
 {'MCQ': 'What is a limitation of the Production Possibility Cu

In [28]:
quiz = pd.DataFrame(quiz_table_data)

In [29]:
quiz.to_csv("quiz.csv", index=False)